# 台股財報擷取-資產負債表

網址：[公開資訊觀測站/財務報表/採IFRS後/合併/個別報表/資產負債表](https://mops.twse.com.tw/mops/web/t164sb03)

In [1]:
import requests, sqlite3
import pandas as pd

In [18]:
print('擷取台股財報－資產負債表...')
sid = input('請輸入個股代號：')
year = input('請輸入擷取年度（請輸入民國年，譬如：108）：')
season = input('請輸入擷取季（輸入格式及區間：01 ~ 04）：')

In [3]:
# 設定網址及 post 表單參數

url = 'https://mops.twse.com.tw/mops/web/t164sb03'
form_data = {
    'encodeURIComponent':1,
    'step':1,
    'firstin':1,
    'off':1,
    'queryName':'co_id',
    'inputType':'co_id',
    'TYPEK':'all',
    'isnew':'false',
    'co_id':sid,
    'year':year,
    'season':season,
}

In [20]:
# 擷取資料

try:
    data = requests.post(url, data=form_data)
except Exception as e:
    print('**WARRN: 擷取失敗')
    print('錯誤訊息', e)
else:
    print('成功擷取！')
    #print(data.text[:1000])

In [21]:
# 使用 pandas 讀取網頁裡的表格資料

dfs = pd.read_html(data.text)
#dfs = pd.read_html('i1-bs.html')

In [22]:
# 取得資產負債表表格資料

# print(len(dfs))
# dfs[10]
df = dfs[10]
df.head(5)

In [23]:
# 查看列數及欄數

df.shape
#df.shape[1]

In [24]:
# 重設欄位名稱

#print(df.columns)
df.columns = list(range(df.shape[1]))
#print(df.columns)
df.head(5)

In [25]:
# 設定索引值

df = df.set_index(0)
df.head(5)

In [26]:
# 只取第一欄資料(即當季財報), 及刪除 nan 項

df1 = df[1].dropna()
df1.head(10)

In [27]:
# 只取1個欄位資料後, 變成 series, 需轉換為 Dataframe

print('轉換前:', type(df1))
df1 = pd.DataFrame(df1)
print('轉換後:', type(df1))

In [28]:
# 轉置表格, 即列與欄交換

df1 = df1.transpose()
df1

In [29]:
# 刪除重覆欄位

df1 = df1.loc[:, ~df1.columns.duplicated()]
df1

In [30]:
# 新增股票代碼及財報日期欄位

df1['sid'] = sid
df1['year'] = year
df1['season'] = season

# 設定索引值

df1 = df1.set_index(['sid', 'year', 'season'])
df1

In [31]:
# 資料庫儲存位置

import os

filename = os.path.join('data', 'stock.db')
filename

In [32]:
#conn = sqlite3.connect(os.path.join('data', 'stock.db'))
conn = sqlite3.connect(filename)

# 判斷 bs table 是否已存在
query = "select count(*) from sqlite_master where type='table' and name='bs'"
exist = list(conn.execute(query))[0][0]

# 如果 bs table 存在，新增資料，否則，建立表格及新增資料
if exist:
    # 讀取資料庫中原有資料
    data = pd.read_sql('select * from bs', conn, index_col=(['sid', 'year', 'season']))
    
    # 合併原有資料與新資料
    newdf = pd.concat([data, df1], sort=True)

    # 刪除重覆項
    newdf.drop_duplicates(inplace=True)
    
    newdf.to_sql('bs', conn, if_exists='replace')
    print('資料新增完畢!')
else:
    df1.to_sql('bs', conn, if_exists='replace')
    print('資料儲存完畢!')

conn.close()